In [ ]:
!pip install simpletransformers

In [ ]:
#!pip install tensorflow_text
#import tensorflow_text as text
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.layers import *
import tensorflow_hub as hub
from keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
def CapsNet(input_shape, n_class, routings):
   x = layers.Input(shape=input_shape)

   # Layer 1: Just a conventional Conv2D layer
   conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

   # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
   primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

   # Layer 3: Capsule layer. Routing algorithm works here.
   digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
   name='digitcaps')(primarycaps)

   # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
   # If using tensorflow, this will not be necessary. :)
   out_caps = Length(name='capsnet')(digitcaps)

   # Decoder network.
   y = layers.Input(shape=(n_class,))
   masked_by_y = Mask()([digitcaps, y]) # The true label is used to mask the output of capsule layer. For training
   masked = Mask()(digitcaps) # Mask using the capsule with maximal length. For prediction

   # Shared Decoder model in training and prediction
   decoder = models.Sequential(name='decoder')
   decoder.add(layers.Dense(512, activation='relu', input_dim=16*n_class))
   decoder.add(layers.Dense(1024, activation='relu'))
   decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
   decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

   # Models for training and evaluation (prediction)
   train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
   eval_model = models.Model(x, [out_caps, decoder(masked)])

   # manipulate model
   noise = layers.Input(shape=(n_class, 16))
   noised_digitcaps = layers.Add()([digitcaps, noise])
   masked_noised_y = Mask()([noised_digitcaps, y])
   manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))

   return train_model, eval_model, manipulate_model

In [ ]:
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [ ]:
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 8
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=True,
)

In [ ]:
df = pd.read_csv('file.csv')

In [ ]:
df = df.astype(str)

In [ ]:
df = df[['Patient dialogue', 'Description']]
df.columns = ['input_text', 'target_text']

In [ ]:
trn_df = df[:8000]
val_df = df[8000:]

In [ ]:
# Train the model
model.train_model(trn_df, eval_data=val_df)

  0%|          | 0/8000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

  0%|          | 0/2049 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/2049 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/257 [00:00<?, ?it/s]

(8000,
 {'eval_loss': [1.1053499052960585,
   1.0610810724677742,
   1.0610810724677742,
   1.0785744744516068,
   1.1043792151291547,
   1.1043792151291547,
   1.1805278814256424,
   1.2509954098597575,
   1.2509954098597575,
   1.3268249535838919,
   1.4082996717686782,
   1.4082996717686782],
  'global_step': [1000,
   2000,
   2000,
   3000,
   4000,
   4000,
   5000,
   6000,
   6000,
   7000,
   8000,
   8000],
  'train_loss': [0.9878384470939636,
   1.1325980424880981,
   1.1325980424880981,
   0.8613367080688477,
   0.5536429286003113,
   0.5536429286003113,
   0.5238649845123291,
   0.418341726064682,
   0.418341726064682,
   0.33400434255599976,
   0.26486048102378845,
   0.26486048102378845]})

In [ ]:
val_df = val_df.reset_index()

In [ ]:
generated=[]
actual=[]
patient_dialogue=[]
for i in range(len(val_df)):
  actual.append(val_df['target_text'][i])
  patient_dialogue.append(val_df['input_text'][i])
  pred = model.predict([val_df['input_text'][i]])[0]
  print(pred)
  print(val_df['target_text'][i])
  generated.append(pred)

In [ ]:
file = pd.DataFrame(list(zip(patient_dialogue, actual, generated)),
               columns =['Input text', 'Actual', 'Generated'])

In [ ]:
file.to_csv('generated.csv')